In [20]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine, text
import urllib.parse

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import os
from matplotlib.backends.backend_pdf import PdfPages

In [27]:
reference_date = 202407
ratio_threshold = 20.0
this_cycle = 2517
this_version='A01'

# Connection parameters - UPDATE WITH YOUR SERVER NAME
server = 'FRCDBApppr'      # e.g., 'localhost' or 'server.domain.com'
database = 'Med_Forecast'


# Create SQLAlchemy connection string for Windows Authentication
connection_string = f'mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes'
engine = create_engine(connection_string)
    
# SQL query (parameterized ThisCycle)
query = """
SELECT ForecastMeg,	ForecastSvc,	MonthOfService,PerCap	
FROM dbo.trackingDetail_history
WHERE ThisCycle = :this_cycle and ReleaseVerLetter = :this_version
"""

# Execute query and load data into pandas DataFrame
stmt = text(query)
df = pd.read_sql(stmt, engine, params={"this_cycle": this_cycle,"this_version":this_version })

In [28]:


# Convert MonthOfService to datetime for easier manipulation
df['MonthOfService'] = pd.to_datetime(df['MonthOfService'], format='%Y%m')

# Convert integer YYYYMM reference_date to pandas Timestamp (first day of month)
reference_date = pd.to_datetime(str(reference_date), format='%Y%m')

# Calculate 6-month periods
six_months_before = reference_date - pd.DateOffset(months=6)
six_months_after = reference_date + pd.DateOffset(months=6)

print(f"Reference date: {reference_date.strftime('%Y-%m')}")
print(f"6 months before: {six_months_before.strftime('%Y-%m')} to {(reference_date - pd.DateOffset(days=1)).strftime('%Y-%m')}")
print(f"6 months after: {(reference_date + pd.DateOffset(days=1)).strftime('%Y-%m')} to {six_months_after.strftime('%Y-%m')}")
print(f"Ratio threshold: {ratio_threshold}")

# Group by ForecastMeg and ForecastSvc (cell)
analysis_records = []

EPSILON = 1e-12

for (forecast_meg, forecast_svc), cell_data in df.groupby(['ForecastMeg', 'ForecastSvc']):
    # Filter data for 6 months before reference
    before_data = cell_data[
        (cell_data['MonthOfService'] >= six_months_before) &
        (cell_data['MonthOfService'] < reference_date)
    ]

    # Filter data for 6 months after reference
    after_data = cell_data[
        (cell_data['MonthOfService'] > reference_date) &
        (cell_data['MonthOfService'] <= six_months_after)
    ]

    if len(before_data) > 0 and len(after_data) > 0:
        before_avg = before_data['PerCap'].mean()
        after_avg = after_data['PerCap'].mean()

        ratio = None
        reason = None

        if abs(before_avg) < EPSILON and abs(after_avg) < EPSILON:
            # both zero -> skip (no plot)
            reason = None
        elif abs(before_avg) < EPSILON and abs(after_avg) >= EPSILON:
            # before zero, after non-zero -> plot
            reason = 'after_nonzero_before_zero'
        elif before_avg > EPSILON:
            ratio = after_avg / before_avg
            if ratio > ratio_threshold:
                reason = 'ratio_gt_threshold'

        analysis_records.append({
            'ForecastMeg': forecast_meg,
            'ForecastSvc': forecast_svc,
            'BeforeAvg': before_avg,
            'AfterAvg': after_avg,
            'Ratio': ratio,
            'BeforeCount': len(before_data),
            'AfterCount': len(after_data),
            'Reason': reason
        })

# Convert to DataFrame for easier analysis
analysis_df = pd.DataFrame(analysis_records)

# Cells flagged for plotting per rules
flagged_cells = analysis_df[analysis_df['Reason'].notna()].copy()

print(f"\nTotal cells analyzed: {len(analysis_df)}")
print(f"Cells flagged for charts: {len(flagged_cells)}")

if len(flagged_cells) > 0:
    # Create one PDF with all graphs (timestamped to avoid file locks)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    pdf_filename = f'high_ratio_cells_analysis_{timestamp}.pdf'

    with PdfPages(pdf_filename) as pdf:
        # Create individual charts for each flagged cell
        for idx, (_, cell) in enumerate(flagged_cells.iterrows()):
            forecast_meg = cell['ForecastMeg']
            forecast_svc = cell['ForecastSvc']

            # Get all data for this cell
            cell_data = df[
                (df['ForecastMeg'] == forecast_meg) &
                (df['ForecastSvc'] == forecast_svc)
            ].sort_values('MonthOfService')

            # Create the plot
            fig, ax = plt.subplots(figsize=(12, 8))

            # Plot PerCap over time
            ax.plot(
                cell_data['MonthOfService'],
                cell_data['PerCap'],
                marker='o',
                linewidth=2,
                markersize=6,
                color='blue'
            )

            # Add vertical line for reference date
            ax.axvline(
                x=reference_date,
                color='red',
                linestyle='--',
                linewidth=2,
                label=f'Reference: {reference_date.strftime("%Y-%m")}'
            )

            # Add horizontal lines for averages
            ax.axhline(
                y=cell['BeforeAvg'],
                color='blue',
                linestyle=':',
                linewidth=2,
                label=f'Before Avg: {cell["BeforeAvg"]:.2f}'
            )
            ax.axhline(
                y=cell['AfterAvg'],
                color='green',
                linestyle=':',
                linewidth=2,
                label=f'After Avg: {cell["AfterAvg"]:.2f}'
            )

            ratio_text = f"{cell['Ratio']:.2f}" if pd.notna(cell['Ratio']) else 'N/A'
            ax.set_title(
                f'PerCap Over Time - {forecast_meg} / {forecast_svc}\nRatio: {ratio_text} | Rule: {cell["Reason"]}',
                fontsize=14,
                fontweight='bold'
            )
            ax.set_xlabel('Month of Service', fontsize=12)
            ax.set_ylabel('PerCap', fontsize=12)
            ax.legend()
            ax.grid(True, alpha=0.3)

            # Format x-axis dates
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
            ax.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
            plt.setp(ax.xaxis.get_majorticklabels(), rotation=45)

            plt.tight_layout()

            # Save to PDF
            pdf.savefig(fig, bbox_inches='tight')
            plt.close()

            print(f"Added chart for {forecast_meg} / {forecast_svc} to PDF (Rule: {cell['Reason']})")

    print(f"\nAll charts saved in single PDF: {pdf_filename}")
else:
    print("\nNo cells met the criteria for charting")

Reference date: 2024-07
6 months before: 2024-01 to 2024-06
6 months after: 2024-07 to 2025-01
Ratio threshold: 20.0

Total cells analyzed: 615
Cells flagged for charts: 16
Added chart for 1212 / 751 to PDF (Rule: after_nonzero_before_zero)
Added chart for 1230 / 385 to PDF (Rule: after_nonzero_before_zero)
Added chart for 1251 / 221 to PDF (Rule: after_nonzero_before_zero)
Added chart for 1261 / 221 to PDF (Rule: after_nonzero_before_zero)
Added chart for 1261 / 223 to PDF (Rule: after_nonzero_before_zero)
Added chart for 1271 / 221 to PDF (Rule: after_nonzero_before_zero)
Added chart for 1280 / 221 to PDF (Rule: after_nonzero_before_zero)
Added chart for 1330 / 680 to PDF (Rule: after_nonzero_before_zero)
Added chart for 1495 / 410 to PDF (Rule: after_nonzero_before_zero)
Added chart for 1671 / 761 to PDF (Rule: after_nonzero_before_zero)
Added chart for 1676 / 385 to PDF (Rule: after_nonzero_before_zero)
Added chart for 1861 / 223 to PDF (Rule: ratio_gt_threshold)
Added chart for 18